In [1]:
import os
import glob
folder_path = "/Users/joseph/Desktop/UCSD/25spring/lean-playground/minif2f-solutions/test"

lean_files = glob.glob(os.path.join(folder_path, "*.lean"))
lean_files = sorted(lean_files, key=lambda x: os.path.basename(x))
# for file_path in lean_files:
#     print(os.path.basename(file_path))

In [2]:
# fp = "/Users/joseph/Desktop/UCSD/25spring/lean-playground/minif2f-solutions/test/aime_1983_p1.lean"
# with open(fp, "r") as f:
#     raw = f.read()
# spos = raw.index("theorem")
# input_text = raw[spos:]

In [3]:
def parse_indented_lines(lines):
    result = []
    stack = [(-1, result)]  # 顶层是 -1，保证第一个 indent=0 的能挂进去

    for line in lines:
        stripped = line.lstrip()
        if not stripped:
            continue  # 忽略空行
        indent = len(line) - len(stripped)

        node = [stripped]

        # 找到能挂载的父节点
        while stack and indent <= stack[-1][0]:
            stack.pop()

        parent_list = stack[-1][1]
        parent_list.append(node)
        stack.append((indent, node))

    return result[0] if result else []

# lines = input_text.splitlines()
# parsed = parse_indented_lines(lines)
# print(parsed)

In [4]:
# count width (top-level "have"s)
def count_width(arg_parsed):
    width = 0
    for c in arg_parsed:
        if isinstance(c, list):
            width += 1
        elif isinstance(c, str):
            # skip
            continue
        else:
            raise Exception(f"unsupported: {c}")
    return width

In [5]:
# get heights of all top-level "have"s
def count_heights(arg_parsed):
    def get_height(p):
        max_height = 0
        for d in p:
            if isinstance(d, list):
                if d[0].startswith("have"):
                    max_height = max(max_height, get_height(d))
                else:
                    # this may likely be part of the theorem, not "have"
                    # or other indentation like "calc"
                    # skip
                    continue
            elif isinstance(d, str):
                # skip
                max_height = max(max_height, 1)
            else:
                raise Exception(f"unsupported: {d}")
        # "have" itself is 1
        return 1 + max_height
                
    heights = []
    for c in arg_parsed:
        if isinstance(c, list):
            if c[0].startswith("have"):
                heights.append(get_height(c))
            else:
                # this may likely be part of the theorem, not "have"
                # or other indentation like "calc"
                # skip
                continue
        elif isinstance(c, str):
            # not "have", don't count
            continue
        else:
            raise Exception(f"unsupported: {c}")

    return heights

In [13]:
import statistics
from collections import Counter

counter = Counter([])

print(f"benchmark, width, min_height, max_height, avg_height, total_haves")
for file_path in lean_files:
    with open(file_path, "r") as f:
        raw = f.read()
    spos = raw.index("theorem")
    input_text = raw[spos:]

    lines = input_text.splitlines()
    parsed = parse_indented_lines(lines)
    ww = count_width(parsed)
    hhs = count_heights(parsed)
    print(f"{os.path.basename(file_path)}, {ww}, {min(hhs)}, {max(hhs)}, {statistics.mean(hhs):.2f}, {raw.count('have ')}")

    counter += Counter(raw.split())
    # break

benchmark, width, min_height, max_height, avg_height, total_haves
aime_1983_p1.lean, 19, 2, 5, 2.94, 72
aime_1983_p2.lean, 10, 2, 3, 2.75, 17
aime_1983_p3.lean, 8, 2, 5, 4.00, 33
aime_1984_p1.lean, 6, 2, 3, 2.67, 15
aime_1984_p7.lean, 196, 3, 3, 3.00, 574
aime_1987_p5.lean, 7, 3, 11, 5.00, 81
aime_1988_p8.lean, 14, 3, 5, 3.82, 67
aime_1989_p8.lean, 7, 3, 4, 3.50, 7
aime_1990_p15.lean, 7, 3, 4, 3.50, 16
aime_1990_p4.lean, 6, 4, 5, 4.50, 18
aime_1991_p9.lean, 11, 2, 4, 2.78, 30
aime_1994_p3.lean, 34, 3, 4, 3.94, 317
aime_1997_p9.lean, 9, 2, 5, 3.83, 44
aime_1999_p11.lean, 11, 2, 8, 4.38, 70
algebra_2varlineareq_fp3zeq11_3tfm1m5zeqn68_feqn10_zeq7.lean, 11, 2, 4, 3.00, 30
algebra_9onxpypzleqsum2onxpy.lean, 14, 2, 3, 2.25, 27
algebra_abpbcpcageq3_sumaonsqrtapbgeq3onsqrt2.lean, 11, 3, 4, 3.62, 79
algebra_absapbon1pabsapbleqsumabsaon1pabsa.lean, 7, 2, 5, 3.20, 50
algebra_absxm1pabsxpabsxp1eqxp2_0leqxleq1.lean, 5, 2, 3, 2.67, 20
algebra_amgm_sum1toneqn_prod1tonleq1.lean, 4, 3, 3, 3.00, 36
alge

In [16]:
counter

Counter({':': 9497,
         ':=': 6558,
         'have': 6180,
         'by': 5479,
         '*': 5218,
         '=': 5121,
         '+': 4647,
         '<;>': 3645,
         '2': 2743,
         '^': 2511,
         '/': 2150,
         '-': 2087,
         'a': 1930,
         'rw': 1869,
         'norm_num': 1801,
         '0': 1761,
         'f': 1583,
         'linarith': 1575,
         'x': 1541,
         'ℝ)': 1537,
         '1': 1448,
         'at': 1313,
         '3': 1194,
         'n': 1057,
         'b': 1050,
         'exact': 943,
         '≤': 830,
         '(2': 780,
         '<': 773,
         'Real.sqrt': 704,
         'ring_nf': 696,
         '4': 695,
         'nlinarith': 687,
         'the': 672,
         'k': 656,
         '(try': 653,
         '⊢': 643,
         'Real.log': 627,
         '(x': 594,
         'omega': 588,
         '--': 588,
         '(a': 580,
         'y': 561,
         'h₁': 553,
         'Real.pi': 552,
         '5': 522,
         '%': 515,
     